In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import os
import requests
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier

LOAD THE DATA SET

In [ ]:
# URL to the raw version of the dataset on GitHub
url = 'https://raw.githubusercontent.com/mkjubran/ENCS5141Datasets/main/BankNotesDataset.csv'

file_path = r'C:\Users\nsrha\Downloads\BankNotesDataset.csv'
response = requests.get(url)
# Check if the request was successful
if response.status_code == 200:
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print("Dataset downloaded successfully!")
else:
    print("Failed to download the dataset. Status code:", response.status_code)
# Check if the file exists
if os.path.exists(file_path):
    print("File exists at the specified path.")
    df = pd.read_csv(file_path)
    print(df.head())
else:
    print("File does not exist at the specified path. Please check the download steps.")


In [ ]:
# Basic info and data types
print(df.info())

# Summary statistics for numeric features
print(df.describe())

# Check for any missing values
print(df.isnull().sum())

# Unique values in categorical fields
print("Unique values in 'Currency':", df['Currency'].unique())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24826 entries, 0 to 24825
Columns: 260 entries, Unnamed: 0 to Orientation
dtypes: float64(256), int64(3), object(1)
memory usage: 49.2+ MB
None
         Unnamed: 0           v_0           v_1           v_2           v_3  \
count  24826.000000  24826.000000  24826.000000  24826.000000  24826.000000   
mean   12412.500000      1.212995      1.204855      1.614074      1.603912   
std     7166.793228      1.460750      1.417384      1.578595      1.633488   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%     6206.250000      0.000000      0.000000      0.000000      0.000000   
50%    12412.500000      0.643254      0.703254      1.276952      1.247524   
75%    18618.750000      2.092841      2.041682      2.751267      2.593737   
max    24825.000000     10.361509      9.754471      7.938003      9.515128   

                v_4           v_5           v_6           v_7           v_8  \
count  24826.00

In [ ]:
# Print the column labels
print("Column labels of the DataFrame:")
print(df.columns.tolist())

In [ ]:
# Check if the 'Orientation' column exists
if 'orientation' in df.columns:
    print("The column 'Orientation' exists in the DataFrame.")
else:
    print("The column 'Orientation' does not exist in the DataFrame.")

In [ ]:
# Inspect unique values in the 'Denomination' column
print("Unique values in 'Denomination':", df['Denomination'].unique())


Given the variety of formats in the "Denomination" column, which combines both denomination and orientation, and sometimes includes additional segments (e.g., "10_1_1"), we need to refine our approach to correctly split and extract the relevant information.


I NEED TO SPLIT THE denomination and orientation

In [ ]:
# Function to split denomination and orientation
def split_denomination_orientation(value):
    parts = re.split(r'_(?=\d+$)', value)  # Split on the last underscore before digits only
    if len(parts) == 2:
        return parts[0], parts[1]
    else:
        return parts[0], None  # Default orientation to None if not present

# Apply the function to each entry in the Denomination column
df[['Denomination', 'Orientation']] = df['Denomination'].apply(
    lambda x: pd.Series(split_denomination_orientation(x))
)

# Convert the new 'Denomination' and 'Orientation' columns to appropriate data types
df['Denomination'] = df['Denomination'].astype(int)
df['Orientation'] = pd.to_numeric(df['Orientation'], errors='coerce')  # Handle cases where orientation might be None

# Verify the changes
print(df[['Denomination', 'Orientation']].head())
print("Unique Denominations:", df['Denomination'].unique())
print("Unique Orientations:", df['Orientation'].unique())

LET'S EXPLORE THE DATA SET

Statistics

In [ ]:
# Summary statistics for all numerical features
print(df.describe())

# Summary statistics for a specific column
print(df['Denomination'].describe())


 Visualization

In [ ]:

# Histogram
df['Denomination'].hist(bins=20)
plt.title('Distribution of Denomination')
plt.show()


In [ ]:

# Box plot
sns.boxplot(x='Denomination', data=df)
plt.title('Box Plot of Denomination')
plt.show()

converting the "Currency" column from a string to a numeric data type is essential for performing correlation analysis, as correlation coefficients require numerical input. There are several ways to encode categorical data like currency names into numerical format, each suitable for different scenarios:

Label Encoding
One-Hot Encoding

apply one HOT ENCODING

In [ ]:
df_one_hot = pd.get_dummies(df, columns=['Currency'], prefix='Currency')
# Check the new DataFrame with one-hot encoded columns
print(df_one_hot.head())


Use in Correlation Heatmap For correlation purposes, one-hot encoding is often more appropriate because it avoids implying any ordinal relationship between categories. Here’s how you could use it when creating a correlation heatmap



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df_one_hot is the DataFrame with one-hot encoded 'Currency' columns
correlation_matrix = df_one_hot.corr()

# Create a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()



# Now, calculate the correlation matrix
correlation_matrix = df_one_hot.corr()

In [ ]:

correlation_matrix.style.background_gradient(cmap='coolwarm')
df_encoded = pd.get_dummies(df, columns=['Currency'])

# Calculate the correlation matrix
correlation_matrix = df_encoded.corr()

# Function to apply custom styles
def highlight_significant(val):
    color = 'red' if abs(val) > 0.5 else 'black'
    return f'color: {color}'

# Styling the DataFrame
styled_matrix = correlation_matrix.style.background_gradient(cmap='coolwarm') \
    .applymap(highlight_significant)

# Display styled DataFrame
styled_matrix


CHECK FOR MISSING VALUES

In [ ]:
# Check for missing values
print(df_one_hot.isnull().sum)


i need to APPLY DIMENSIONALITY REDUCTION

In [ ]:
target_columns = ['Orientation', 'Denomination'] + [col for col in df_one_hot.columns if col.startswith('Currency_')]
X = df_one_hot.drop(target_columns, axis=1)
# Combine all target columns for y
y = df_one_hot[target_columns]
# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize PCA, choosing to preserve 95% of the variance
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

# Output the results of PCA dimension reduction
print("Original number of features:", X.shape[1])
print("Reduced number of features:", X_pca.shape[1])

Original number of features: 257
Reduced number of features: 125


APPLY FEATURE SELECTION

In [ ]:
selected_features = np.zeros(X_pca.shape[1], dtype=int)  # Array to store counts of feature selection

for column in y.columns:
    selector = SelectKBest(score_func=f_classif, k=40)
    y_label = y[column]
    selector.fit(X_pca, y_label)
    selected_features += selector.get_support().astype(int)  # Sum up selections across labels

# Determine the most frequently selected features
feature_indices = np.where(selected_features > 0)[0]
# Reduce X to the most frequently selected features
X_selected = X_pca[:, feature_indices]
print("Total selections for each feature across all labels:")
print(selected_features)
print("Indices of selected features (most frequently selected):")
print(feature_indices)
print("Number of features selected based on threshold:")
print(len(feature_indices))
if len(feature_indices) > 0:
    print("Selected features summary:")
    feature_summary = {index: selected_features[index] for index in feature_indices}
    for feature, count in feature_summary.items():
        print(f"Feature index: {feature}, Selection count: {count}")
original_features = X.columns  # Names of original features
full_pca = PCA(n_components=None)
X_full_pca = full_pca.fit_transform(X_scaled)  # X_scaled is your standardized original data
# Get the components that contribute most to variance
components = full_pca.components_
# Analyze the contribution of original features in each selected PCA component
def get_important_features(components, feature_names, component_indices):
    important_features = {}
    for idx in component_indices:
        component = components[idx]
        feature_weights = list(zip(feature_names, component))
        sorted_features = sorted(feature_weights, key=lambda x: abs(x[1]), reverse=True)
        important_features[f"Component {idx + 1}"] = sorted_features[:5]  # Top 5 features contributing to this component
    return important_features

# Using the function to get the most contributing original features for selected components
important_features_dict = get_important_features(components, original_features, feature_indices)
for component, features in important_features_dict.items():
    print(f"{component}:")
    for feature, weight in features:
        print(f"  {feature}, weight: {weight:.4f}")


START WITH MLP
NOW LET'S USE MLP, WITH MULTI CLASSIFIERS AND USING  grid search


1-CURRENCY LABEL CLASSIFIER

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

def perform_grid_search(X_train, y_train, X_test, y_test, label_name):
    parameter_space = {
        'estimator__hidden_layer_sizes': [(50,100,50), (100,)],
        'estimator__activation': ['tanh'],
        'estimator__solver': [ 'adam'],
        'estimator__alpha': [ 0.05,0.1],
        'estimator__learning_rate': ['adaptive'],
    }

    mlp = MLPClassifier(max_iter=5, random_state=42)
    clf = GridSearchCV(OneVsRestClassifier(mlp), parameter_space, n_jobs=-1, cv=3)
    clf.fit(X_train, y_train)

    # Best parameter set
    print(f"Best parameters found for {label_name}: ", clf.best_params_)

    # Predict on test set with best parameters
    y_pred = clf.predict(X_test)
    print(f"Classification report for {label_name}:\n{classification_report(y_test, y_pred)}")

perform_grid_search(X_train, y_train.filter(regex='Currency_'), X_test, y_test.filter(regex='Currency_'), 'Currency')


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686

Best parameters found for Currency:  {'estimator__activation': 'tanh', 'estimator__alpha': 0.1, 'estimator__hidden_layer_sizes': (50, 100, 50), 'estimator__learning_rate': 'adaptive', 'estimator__solver': 'adam'}
Classification report for Currency:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       479
           1       0.98      0.99      0.98       639
           2       0.98      0.93      0.95       355
           3       0.98      0.97      0.98       565
           4       0.91      0.91      0.91       313
           5       0.99      0.99      0.99       378
           6       0.90      0.90      0.90       570
           7       0.98      0.98      0.98       500
           8       0.92      0.89      0.90       321
           9       1.00      0.98      0.99       362
          10       0.92      0.72      0.81       272
          11       0.94      0.93      0.94       338
          12       1.00      1.00      1.00     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2-Denomation calssifier

In [ ]:
def perform_grid_search_denomination(X_train, y_train, X_test, y_test):
    parameter_space = {
        'hidden_layer_sizes': [(50,100,100)],
        'activation': ['tanh'],
        'solver': [ 'sgd','adam'],
        'alpha': [0.1],
        'learning_rate': ['adaptive'],
    }

    mlp = MLPClassifier(max_iter=25, random_state=42)
    clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
    clf.fit(X_train, y_train)

    print("Best parameters found for Denomination:", clf.best_params_)

    y_pred = clf.predict(X_test)
    print("Classification Report for Denomination:\n", classification_report(y_test, y_pred))


X_train, X_test, y_train_denomination, y_test_denomination = train_test_split(X_pca, df_one_hot['Denomination'], test_size=0.3, random_state=42)
perform_grid_search_denomination(X_train, y_train_denomination, X_test, y_test_denomination)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (25) reached and the optimization hasn't converged yet.
  warnings.warn(


Best parameters found for Denomination: {'activation': 'tanh', 'alpha': 0.1, 'hidden_layer_sizes': (50, 100, 100), 'learning_rate': 'adaptive', 'solver': 'adam'}
Classification Report for Denomination:
               precision    recall  f1-score   support

           1       0.93      0.95      0.94       187
           2       0.93      0.95      0.94       240
           5       0.88      0.91      0.90       858
          10       0.93      0.91      0.92       915
          20       0.92      0.92      0.92       907
          50       0.93      0.91      0.92      1034
         100       0.92      0.93      0.93       997
         101       0.91      0.86      0.89        59
         102       0.87      0.86      0.86        90
         200       0.94      0.95      0.95       433
         201       0.75      0.77      0.76        81
         202       0.88      0.79      0.83        90
         500       0.93      0.92      0.93       159
         501       0.58      0.92      0

3- ORIENTATION CLASSIFIER

In [ ]:
def perform_grid_search_orientation(X_train, y_train, X_test, y_test):
    parameter_space = {
        'hidden_layer_sizes': [(50,100,50)],
        'activation': ['tanh'],
        'solver': [ 'sgd','adam'],
        'alpha': [ 0.1],
        'learning_rate': [ 'adaptive'],
    }

    mlp = MLPClassifier(max_iter=15, random_state=42)
    clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
    clf.fit(X_train, y_train)

    print("Best parameters found for Orientation:", clf.best_params_)

    y_pred = clf.predict(X_test)
    print("Classification Report for Orientation:\n", classification_report(y_test, y_pred))


X_train, X_test, y_train_orientation, y_test_orientation = train_test_split(X_pca, df_one_hot['Orientation'], test_size=0.3, random_state=42)
perform_grid_search_orientation(X_train, y_train_orientation, X_test, y_test_orientation)


Best parameters found for Orientation: {'activation': 'tanh', 'alpha': 0.1, 'hidden_layer_sizes': (50, 100, 50), 'learning_rate': 'adaptive', 'solver': 'adam'}
Classification Report for Orientation:
               precision    recall  f1-score   support

           1       0.97      0.94      0.95      3880
           2       0.94      0.96      0.95      3568

    accuracy                           0.95      7448
   macro avg       0.95      0.95      0.95      7448
weighted avg       0.95      0.95      0.95      7448



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


NOW APPLY MLP, BUT WITH MERGING THE THREE LABELS

In [ ]:
def get_currency_from_one_hot(row):
    for col in df_one_hot.columns:
        if col.startswith('Currency_') and row[col] == 1:
            return col.split('_')[1]
    return None  # Return None or an appropriate default if no currency column is active

# Apply the function to create the 'Currency' column
df_one_hot['Currency'] = df_one_hot.apply(get_currency_from_one_hot, axis=1)
# Combine labels into a single string for the combined label
def create_combined_label(row):
    return f"{row['Currency']}_{row['Denomination']}_{row['Orientation']}"

df_one_hot['combined_label'] = df_one_hot.apply(create_combined_label, axis=1)

# Prepare the data
X = df_one_hot.drop(['Currency', 'Denomination', 'Orientation', 'combined_label'] + [col for col in df_one_hot.columns if col.startswith('Currency_')], axis=1)
y = df_one_hot['combined_label']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Setup Grid Search for MLP on the Combined Label
def perform_grid_search_combined_label(X_train, y_train, X_test, y_test):
    parameter_space = {
        'hidden_layer_sizes': [(50,100,100)],
        'activation': [ 'relu'],
        'solver': [ 'sgd','adam'],
        'alpha': [0.1],
        'learning_rate': ['adaptive'],
    }

    mlp = MLPClassifier(max_iter=5, random_state=42)
    clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
    clf.fit(X_train, y_train)

    print("Best parameters found for combined label:", clf.best_params_)

    y_pred = clf.predict(X_test)
    print("Classification Report for Combined Label:\n", classification_report(y_test, y_pred))

# Execute the grid search
perform_grid_search_combined_label(X_train, y_train, X_test, y_test)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best parameters found for combined label: {'activation': 'relu', 'alpha': 0.1, 'hidden_layer_sizes': (50, 100, 100), 'learning_rate': 'adaptive', 'solver': 'adam'}
Classification Report for Combined Label:
               precision    recall  f1-score   support

   AUD_100_1       0.93      0.85      0.89        46
   AUD_100_2       0.62      0.90      0.74        51
    AUD_10_1       0.78      0.86      0.82        57
    AUD_10_2       0.91      0.66      0.77        47
    AUD_20_1       0.62      0.61      0.62        41
    AUD_20_2       0.57      0.52      0.54        48
    AUD_50_1       1.00      0.74      0.85        46
    AUD_50_2       0.79      0.98      0.88        43
     AUD_5_1       0.90      0.95      0.92        58
     AUD_5_2       0.89      0.93      0.91        42
   BRL_100_1       0.93      0.90      0.92        62
   BRL_100_2       0.83      0.90      0.86        21
    BRL_10_1       0.88      1.00      0.94        36
    BRL_10_2       0.98      1.00   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NOW APPLY MLP BUT WITHOUT PCA, USE 3 CLASSIFIERS

1-ORIENTATION

In [ ]:
def perform_grid_search_orientation(X_train, y_train, X_test, y_test):
    parameter_space = {
        'hidden_layer_sizes': [(50,100,50)],
        'activation': ['tanh'],
        'solver': [ 'sgd','adam'],
        'alpha': [ 0.1],
        'learning_rate': [ 'adaptive'],
    }

    mlp = MLPClassifier(max_iter=5, random_state=42)
    clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
    clf.fit(X_train, y_train)

    print("Best parameters found for Orientation:", clf.best_params_)

    y_pred = clf.predict(X_test)
    print("Classification Report for Orientation:\n", classification_report(y_test, y_pred))

X_train, X_test, y_train_orientation, y_test_orientation = train_test_split(X_scaled, df_one_hot['Orientation'], test_size=0.3, random_state=42)
perform_grid_search_orientation(X_train, y_train_orientation, X_test, y_test_orientation)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


Best parameters found for Orientation: {'activation': 'tanh', 'alpha': 0.1, 'hidden_layer_sizes': (50, 100, 50), 'learning_rate': 'adaptive', 'solver': 'adam'}
Classification Report for Orientation:
               precision    recall  f1-score   support

           1       0.95      0.95      0.95      3880
           2       0.95      0.94      0.94      3568

    accuracy                           0.95      7448
   macro avg       0.95      0.95      0.95      7448
weighted avg       0.95      0.95      0.95      7448



2-DENOMINATION

In [ ]:
def perform_grid_search_denomination(X_train, y_train, X_test, y_test):
    parameter_space = {
        'hidden_layer_sizes': [(50, 100, 50)],
        'activation': ['tanh'],
        'solver': ['sgd','adam'],
        'alpha': [0.1],
        'learning_rate': ['adaptive'],
    }

    mlp = MLPClassifier(max_iter=5, random_state=42)
    clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
    clf.fit(X_train, y_train)

    print("Best parameters found for Denomination:", clf.best_params_)

    y_pred = clf.predict(X_test)
    print("Classification Report for Denomination:\n", classification_report(y_test, y_pred))

# Splitting the data for Denomination
X_train, X_test, y_train_denomination, y_test_denomination = train_test_split(X_scaled, df_one_hot['Denomination'], test_size=0.3, random_state=42)
perform_grid_search_denomination(X_train, y_train_denomination, X_test, y_test_denomination)


Best parameters found for Denomination: {'activation': 'tanh', 'alpha': 0.1, 'hidden_layer_sizes': (50, 100, 50), 'learning_rate': 'adaptive', 'solver': 'adam'}
Classification Report for Denomination:
               precision    recall  f1-score   support

           1       0.87      0.91      0.89       187
           2       0.85      0.95      0.90       240
           5       0.86      0.90      0.88       858
          10       0.87      0.91      0.89       915
          20       0.92      0.88      0.90       907
          50       0.91      0.89      0.90      1034
         100       0.92      0.91      0.91       997
         101       0.78      0.80      0.79        59
         102       0.74      0.82      0.78        90
         200       0.90      0.96      0.93       433
         201       0.73      0.75      0.74        81
         202       0.93      0.72      0.81        90
         500       0.90      0.83      0.87       159
         501       0.86      0.50      0.

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


3-CURRENCY

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
y_currency = df_one_hot.filter(regex='^Currency_')

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train_currency, y_test_currency = train_test_split(X_scaled, y_currency, test_size=0.3, random_state=42)

# Define the Grid Search function for the Currency classifier
def perform_grid_search_currency(X_train, y_train, X_test, y_test):
    parameter_space = {
        'estimator__hidden_layer_sizes': [(50, 100, 50), (50,)],
        'estimator__activation': ['tanh'],
        'estimator__solver': ['sgd', 'adam'],
        'estimator__alpha': [0.1, 1],
        'estimator__learning_rate': ['adaptive'],
    }

    mlp = MLPClassifier(max_iter=5, random_state=42)  # Increased max_iter for better convergence
    clf = GridSearchCV(OneVsRestClassifier(mlp), parameter_space, n_jobs=-1, cv=3)
    clf.fit(X_train, y_train)

    print("Best parameters found for Currency:", clf.best_params_)

    y_pred = clf.predict(X_test)
    print("Classification Report for Currency:\n", classification_report(y_test, y_pred))

# Execute the grid search
perform_grid_search_currency(X_train, y_train_currency, X_test, y_test_currency)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686

Best parameters found for Currency: {'estimator__activation': 'tanh', 'estimator__alpha': 0.1, 'estimator__hidden_layer_sizes': (50, 100, 50), 'estimator__learning_rate': 'adaptive', 'estimator__solver': 'adam'}
Classification Report for Currency:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       479
           1       0.99      0.99      0.99       639
           2       0.97      0.94      0.96       355
           3       0.99      0.96      0.97       565
           4       0.88      0.90      0.89       313
           5       0.99      0.99      0.99       378
           6       0.89      0.91      0.90       570
           7       0.98      0.98      0.98       500
           8       0.92      0.88      0.90       321
           9       0.99      0.97      0.98       362
          10       0.90      0.71      0.79       272
          11       0.95      0.91      0.93       338
          12       1.00      1.00      1.00     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NOW APPLY MLP, NO PCA, BUT WITH ONE CLASSIFIER

In [ ]:
def get_currency_from_one_hot(row):
    for col in df_one_hot.columns:
        if col.startswith('Currency_') and row[col] == 1:
            return col.split('_')[1]
    return None  # Return None or an appropriate default if no currency column is active

# Apply the function to create the 'Currency' column
df_one_hot['Currency'] = df_one_hot.apply(get_currency_from_one_hot, axis=1)
# Combine labels into a single string for the combined label
def create_combined_label(row):
    return f"{row['Currency']}_{row['Denomination']}_{row['Orientation']}"

df_one_hot['combined_label'] = df_one_hot.apply(create_combined_label, axis=1)

# Prepare the data
X = df_one_hot.drop(['Currency', 'Denomination', 'Orientation', 'combined_label'] + [col for col in df_one_hot.columns if col.startswith('Currency_')], axis=1)
y = df_one_hot['combined_label']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Setup Grid Search for MLP on the Combined Label
def perform_grid_search_combined_label(X_train, y_train, X_test, y_test):
    parameter_space = {
        'hidden_layer_sizes': [(50,100,100)],
        'activation': [ 'relu'],
        'solver': [ 'sgd','adam'],
        'alpha': [0.1],
        'learning_rate': ['adaptive'],
    }

    mlp = MLPClassifier(max_iter=5, random_state=42)
    clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
    clf.fit(X_train, y_train)

    print("Best parameters found for combined label:", clf.best_params_)

    y_pred = clf.predict(X_test)
    print("Classification Report for Combined Label:\n", classification_report(y_test, y_pred))

# Execute the grid search
perform_grid_search_combined_label(X_train, y_train, X_test, y_test)


NO LET'S START WITH SVM

1-SVM, PCA, MULTI CLASSIFIERS

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
df_one_hot = pd.get_dummies(df, columns=['Currency'], prefix='Currency')

# Prepare features X by excluding all target-related columns
target_columns = ['Orientation', 'Denomination'] + [col for col in df_one_hot.columns if col.startswith('Currency_')]
X = df_one_hot.drop(target_columns, axis=1)

# Combine all target columns for y
y = df_one_hot[target_columns]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize PCA, choosing to preserve 95% of the variance
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Separate target columns for training
y_train_currency = y_train.filter(like='Currency_')
y_train_denomination = y_train['Denomination']
y_train_orientation = y_train['Orientation']

y_test_currency = y_test.filter(like='Currency_')
y_test_denomination = y_test['Denomination']
y_test_orientation = y_test['Orientation']

# Define the parameter grid
param_grid = {
    'C': [ 1, 10],
    'gamma': [ 'auto'],
    'kernel': [ 'linear','rbf', 'poly']
}

# Function to train and evaluate a classifier for each label in currency
def grid_search_train_evaluate_for_currency(X_train, y_train, X_test, y_test, target_name):
    for label in y_train.columns:
        print(f"Training model for {label}")
        svc = SVC(random_state=42)
        grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)
        grid_search.fit(X_train, y_train[label])

        print(f"Best parameters for {label}: {grid_search.best_params_}")
        y_pred = grid_search.predict(X_test)

        print(f"Classification report for {label}:")
        print(classification_report(y_test[label], y_pred))
        print("Accuracy:", accuracy_score(y_test[label], y_pred))
        print("\n")

# Function to train and evaluate a classifier for single-label targets
def train_evaluate_classifier(X_train, y_train, X_test, y_test, target_name):
    svc = SVC(random_state=42)
    grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=3
                               , scoring='accuracy', verbose=1)
    grid_search.fit(X_train, y_train)

    print(f"Best parameters for {target_name}: {grid_search.best_params_}")
    y_pred = grid_search.predict(X_test)

    print(f"Classification report for {target_name}:")
    print(classification_report(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("\n")

# Perform Grid Search and evaluate each classifier
grid_search_train_evaluate_for_currency(X_train, y_train_currency, X_test, y_test_currency, 'Currency')
train_evaluate_classifier(X_train, y_train_denomination, X_test, y_test_denomination, 'Denomination')
train_evaluate_classifier(X_train, y_train_orientation, X_test, y_test_orientation, 'Orientation')


2-SVM, PCA, SINGLE CLASSIFIER

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
df_one_hot = pd.get_dummies(df, columns=['Currency'], prefix='Currency')

# Prepare features X by excluding target-related columns
target_columns = ['Orientation', 'Denomination'] + [col for col in df_one_hot.columns if col.startswith('Currency_')]
X = df_one_hot.drop(target_columns, axis=1)

# Create a combined label
df_one_hot['Combined_Label'] = df_one_hot.apply(lambda row: f"{row['Orientation']}_{row['Denomination']}_" + '_'.join([f"{col}:{row[col]}" for col in df_one_hot.columns if col.startswith('Currency_') and row[col] == 1]), axis=1)

# Target y with the new combined label
y = df_one_hot['Combined_Label']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize PCA, choosing to preserve 95% of the variance
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Define the parameter grid
param_grid = {
    'C': [ 1, 10],
    'gamma': [ 'auto'],
    'kernel': ['rbf', 'poly','linear']
}

# Function to train and evaluate the classifier
def train_evaluate_classifier(X_train, y_train, X_test, y_test, target_name):
    svc = SVC(random_state=42)
    grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=2, scoring='accuracy', verbose=1)
    grid_search.fit(X_train, y_train)

    print(f"Best parameters for {target_name}: {grid_search.best_params_}")
    y_pred = grid_search.predict(X_test)

    print(f"Classification report for {target_name}:")
    print(classification_report(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("\n")

# Train and evaluate the classifier
train_evaluate_classifier(X_train, y_train, X_test, y_test, 'Combined Label')


Fitting 2 folds for each of 6 candidates, totalling 12 fits
Best parameters for Combined Label: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
Classification report for Combined Label:
                            precision    recall  f1-score   support

1_100000_Currency_IDR:True       1.00      1.00      1.00        21
 1_10000_Currency_JPY:True       1.00      1.00      1.00        32
  1_1000_Currency_IDR:True       0.95      1.00      0.97        18
  1_1000_Currency_INR:True       1.00      1.00      1.00         4
  1_1000_Currency_JPY:True       1.00      0.95      0.98        62
  1_1000_Currency_MXN:True       1.00      1.00      1.00         2
  1_1000_Currency_NNR:True       1.00      0.86      0.92        14
  1_1000_Currency_PHP:True       1.00      1.00      1.00        22
  1_1000_Currency_PKR:True       1.00      1.00      1.00        16
  1_1001_Currency_INR:True       0.73      0.80      0.76        10
  1_1002_Currency_INR:True       1.00      0.82      0.90        11
 

1-SVM, NO PCA, MULTI CLASSIFIERS

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

df_one_hot = pd.get_dummies(df, columns=['Currency'], prefix='Currency')

# Prepare features X by excluding all target-related columns
target_columns = ['Orientation', 'Denomination'] + [col for col in df_one_hot.columns if col.startswith('Currency_')]
X = df_one_hot.drop(target_columns, axis=1)

# Combine all target columns for y
y = df_one_hot[target_columns]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the dataset into training and testing sets without using PCA
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Separate target columns for training
y_train_currency = y_train.filter(like='Currency_')
y_train_denomination = y_train['Denomination']
y_train_orientation = y_train['Orientation']

y_test_currency = y_test.filter(like='Currency_')
y_test_denomination = y_test['Denomination']
y_test_orientation = y_test['Orientation']

# Define the parameter grid
param_grid = {
    'C': [ 1, 10],
    'gamma': [ 'auto'],
    'kernel': ['linear', 'rbf', 'poly']
}

# Function to train and evaluate a classifier for each label in currency
def grid_search_train_evaluate_for_currency(X_train, y_train, X_test, y_test, target_name):
    for label in y_train.columns:
        print(f"Training model for {label}")
        svc = SVC(random_state=42)
        grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=2, scoring='accuracy', verbose=1)
        grid_search.fit(X_train, y_train[label])

        print(f"Best parameters for {label}: {grid_search.best_params_}")
        y_pred = grid_search.predict(X_test)

        print(f"Classification report for {label}:")
        print(classification_report(y_test[label], y_pred))
        print("Accuracy:", accuracy_score(y_test[label], y_pred))
        print("\n")

# Function to train and evaluate a classifier for single-label targets
def train_evaluate_classifier(X_train, y_train, X_test, y_test, target_name):
    svc = SVC(random_state=42)
    grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=2, scoring='accuracy', verbose=1)
    grid_search.fit(X_train, y_train)

    print(f"Best parameters for {target_name}: {grid_search.best_params_}")
    y_pred = grid_search.predict(X_test)

    print(f"Classification report for {target_name}:")
    print(classification_report(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("\n")

# Perform Grid Search and evaluate each classifier
grid_search_train_evaluate_for_currency(X_train, y_train_currency, X_test, y_test_currency, 'Currency')
train_evaluate_classifier(X_train, y_train_denomination, X_test, y_test_denomination, 'Denomination')
train_evaluate_classifier(X_train, y_train_orientation, X_test, y_test_orientation, 'Orientation')


Training model for Currency_AUD
Fitting 2 folds for each of 6 candidates, totalling 12 fits
Best parameters for Currency_AUD: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
Classification report for Currency_AUD:
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      4644
        True       1.00      1.00      1.00       322

    accuracy                           1.00      4966
   macro avg       1.00      1.00      1.00      4966
weighted avg       1.00      1.00      1.00      4966

Accuracy: 0.999597261377366


Training model for Currency_BRL
Fitting 2 folds for each of 6 candidates, totalling 12 fits
Best parameters for Currency_BRL: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
Classification report for Currency_BRL:
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      4525
        True       1.00      0.99      1.00       441

    accuracy                           1.00      4966
   ma

2- SVM, NO PCA, SINGLE CLASSIFIER

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
df_one_hot = pd.get_dummies(df, columns=['Currency'], prefix='Currency')

# Prepare features X by excluding target-related columns
target_columns = ['Orientation', 'Denomination'] + [col for col in df_one_hot.columns if col.startswith('Currency_')]
X = df_one_hot.drop(target_columns, axis=1)

# Create a combined label
df_one_hot['Combined_Label'] = df_one_hot.apply(lambda row: f"{row['Orientation']}_{row['Denomination']}_" + '_'.join([f"{col}:{row[col]}" for col in df_one_hot.columns if col.startswith('Currency_') and row[col] == 1]), axis=1)

# Target y with the new combined label
y = df_one_hot['Combined_Label']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['auto'],
    'kernel': ['rbf', 'poly','linear']
}

# Function to train and evaluate the classifier
def train_evaluate_classifier(X_train, y_train, X_test, y_test, target_name):
    svc = SVC(random_state=42)
    grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=2, scoring='accuracy', verbose=1)
    grid_search.fit(X_train, y_train)

    print(f"Best parameters for {target_name}: {grid_search.best_params_}")
    y_pred = grid_search.predict(X_test)

    print(f"Classification report for {target_name}:")
    print(classification_report(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("\n")

# Train and evaluate the classifier
train_evaluate_classifier(X_train, y_train, X_test, y_test, 'Combined Label')


Fitting 2 folds for each of 9 candidates, totalling 18 fits
Best parameters for Combined Label: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
Classification report for Combined Label:
                            precision    recall  f1-score   support

1_100000_Currency_IDR:True       1.00      1.00      1.00        21
 1_10000_Currency_JPY:True       1.00      1.00      1.00        32
  1_1000_Currency_IDR:True       0.95      1.00      0.97        18
  1_1000_Currency_INR:True       1.00      1.00      1.00         4
  1_1000_Currency_JPY:True       1.00      0.94      0.97        62
  1_1000_Currency_MXN:True       1.00      1.00      1.00         2
  1_1000_Currency_NNR:True       1.00      0.86      0.92        14
  1_1000_Currency_PHP:True       1.00      1.00      1.00        22
  1_1000_Currency_PKR:True       1.00      1.00      1.00        16
  1_1001_Currency_INR:True       0.89      0.80      0.84        10
  1_1002_Currency_INR:True       1.00      0.73      0.84        11
 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
